In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import make_scorer
from scipy import stats
from xgboost import XGBRegressor
import joblib

cols_name = np.load('D:\dataset\dataset\cols_name.npy', allow_pickle=True)
train_df = pd.read_parquet('D:\\dataset\\dataset\\train_data.parquet')

In [ ]:
train_split = train_df.loc[train_df['Moons']<225]
val_split = train_df.loc[train_df['Moons']>=225]
len(train_split), len(val_split)


x_train = train_split.iloc[:, 2:-4].values  
y_train = train_split['target_w'].values    

x_val = val_split.iloc[:, 2:-4].values     
y_val = val_split['target_w'].values         


In [ ]:
groups = train_split['Moons'].values

In [ ]:
print(x_train.shape) 
print(y_train.shape)  

print(x_val.shape)    
print(y_val.shape) 

In [ ]:
def spearman_scorer(y_true, y_pred):
    return stats.spearmanr(y_true, y_pred).correlation

In [ ]:
import numpy as np 
import pandas as pd
from xgboost import XGBRegressor
import joblib



train_df = pd.read_parquet('D:\\dataset\\dataset\\test_df.parquet')


train_split = train_df.loc[train_df['Moons'] < 390]
val_split = train_df.loc[(train_df['Moons'] >= 390) & (train_df['Moons'] <400 )]
test_split = train_df.loc[train_df['Moons'] >= 400]
print(train_split.shape)
print(val_split.shape)
test_split.shape

In [ ]:
def get_top_features(train_split, target_names, n_features=20):
    """
    Lấy các đặc trưng tốt nhất cho mỗi biến mục tiêu.
    """
    top_features = {}
    features_to_exclude = ['target_w', 'target_r', 'target_g', 'target_b']
    
    for target in target_names:
        train_features_only = train_split.drop(columns=features_to_exclude)
        target_corr = train_features_only.corrwith(train_split[target]).reset_index()
        target_corr.columns = ['feature', 'correlation']
        target_corr['correlation'] = abs(target_corr['correlation'])
        top_features[target] = target_corr.sort_values(by='correlation', ascending=False)['feature'][:n_features].tolist()
    
    return top_features


In [ ]:
def get_feature_importances(model, feature_names):
    """
    Lấy điểm quan trọng của các đặc trưng từ mô hình XGBoost.
    """
    importances = model.feature_importances_
    feature_importances = dict(zip(feature_names, importances))
    return feature_importances


In [ ]:
from joblib import load


model_paths = {
    'target_w': 'D:\\dataset\\model_target_w.joblib',
    'target_r': 'D:\\dataset\\model_target_r.joblib',
    'target_g': 'D:\\dataset\\model_target_g.joblib',
    'target_b': 'D:\\dataset\\model_target_b.joblib'
}


target_names = ['target_w', 'target_r', 'target_g', 'target_b']

results = {}


for target in target_names:
    model = load(model_paths[target])
    feature_names = x_train.columns  

  
    feature_importances = get_feature_importances(model, feature_names)

 
    results[target] = {
        'model': model,
        'feature_importances': feature_importances,
        'top_features': get_top_features(train_split, [target])[target]
    }


for target, result in results.items():
    print(f"\nModel saved to: {model_paths[target]}")
    print("\nTop Features:")
    print(result['top_features'])  

    print("\nFeature Importances:")
    for feature, importance in result['feature_importances'].items():
        print(f"{feature}: {importance}")


In [ ]:
import numpy as np 
import pandas as pd
from xgboost import XGBRegressor
import joblib


cols_name = np.load('D:\\dataset\\dataset\\cols_name.npy', allow_pickle=True)
train_df = pd.read_parquet('D:\\dataset\\dataset\\train_data.parquet')


train_split = train_df.loc[train_df['Moons'] < 200]
val_split = train_df.loc[(train_df['Moons'] >= 200) & (train_df['Moons'] < 225)]
test_split = train_df.loc[train_df['Moons'] >= 225]

x_train = train_split.iloc[:, 2:-4]  
y_train = train_split['target_w'].values  

x_val = val_split.iloc[:, 2:-4] 
y_val = val_split['target_w'].values 

x_test = test_split.iloc[:, 2:-4] 
y_test = test_split['target_w'].values  

groups = train_split['Moons'].values

def get_top_features(train_split, target_names, n_features=30):
    """
    Lấy các đặc trưng tốt nhất cho mỗi biến mục tiêu.
    """
    top_features = {}
    features_to_exclude = ['target_w', 'target_r', 'target_g', 'target_b']
    
    for target in target_names:
        train_features_only = train_split.drop(columns=features_to_exclude)
        target_corr = train_features_only.corrwith(train_split[target]).reset_index()
        target_corr.columns = ['feature', 'correlation']
        target_corr['correlation'] = abs(target_corr['correlation'])
        top_features[target] = target_corr.sort_values(by='correlation', ascending=False)['feature'][:n_features].tolist()
    
    return top_features 

def get_feature_importances(model, feature_names):
    """
    Lấy điểm quan trọng của các đặc trưng từ mô hình XGBoost.
    """
    importances = model.feature_importances_
    feature_importances = dict(zip(feature_names, importances))
    sorted_features = sorted(feature_importances.items(), key=lambda item: item[1], ascending=False)
    return sorted_features

def train_and_evaluate(train_split, model_paths):
    
    
    target_names = ['target_w', 'target_r', 'target_g', 'target_b']
    
    
    top_features_dict = get_top_features(train_split, target_names, n_features=130)
    
    
    results = {}
    
    for target in target_names:
        train_top_features = train_split[top_features_dict[target]]
        X = train_top_features.values
        y = train_split[target].values
        
        
        best_params = {
        'learning_rate': 0.006795227920929744,
        'max_depth': 11, 
        'subsample': 0.9005341830239901, 
        'colsample_bytree': 0.39363685180823027, 
        'reg_lambda': 25.490987283296956, 
        'grow_policy': 'lossguide',
        'max_cat_to_onehot': 46,
        'n_estimators': 10000,
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'random_state': 0,
        'enable_categorical': True,
    }
        
        
        model = XGBRegressor(**best_params)
        model.fit(X, y)
        
        
        model_path = model_paths[target]
        joblib.dump(model, model_path)
        
    
        feature_importances = get_feature_importances(model, top_features_dict[target])
        
        results[target] = {
            'model': model,
            'params': best_params,
            'top_features': top_features_dict[target],
            'feature_importances': feature_importances
        }
    
    return results


model_paths = {
    'target_w': 'D:\\dataset\\dataset\\model_target_w.joblib',
    'target_r': 'D:\\dataset\\dataset\\model_target_r.joblib',
    'target_g': 'D:\\dataset\\dataset\\model_target_g.joblib',
    'target_b': 'D:\\dataset\\dataset\\model_target_b.joblib'
}


results = train_and_evaluate(train_split, model_paths)


for target, result in results.items():
    print(f"\nModel saved to: {model_paths[target]}")
    print("\nBest Parameters:")
    for key, value in result['params'].items():
        print(f"{key}: {value}")

    print("\nTop Features:")
    print(result['top_features'])  

    print("\nFeature Importances:")
    for feature, importance in result['feature_importances']:
        print(f"{feature}: {importance}")
